# LangChainでラクラク！「WebChat」
[参考サイト(検索API)](https://zenn.dev/kun432/scraps/cefc75b14ff013)  

# LangChainとは

言語モデルを利用してアプリケーションを開発するためのフレームワークである。  
[公式サイト](https://python.langchain.com/en/latest/index.html)  
[日本語参考サイト](https://zenn.dev/umi_mori/books/prompt-engineer/viewer/langchain_overview)  

In [1]:
# ! pip install langchain google-search-results

In [2]:
from pathlib import Path

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

In [3]:
# GPTのAPIキー読み込み
with open(f'{Path.home()}/.private_info/api_key.txt') as f1:
    api_key = f1.read()

In [4]:
# SerpAPIのキー読み込み
with open(f'{Path.home()}/.private_info/serpapi_key.txt') as f2:
    serpapi_key = f2.read()

# LangChainの「Agents」

言語モデルに渡されたツールを用いて、モデル自体が次にどのようなアクションを取るかを決定し、実行し、観測し、完了するまで繰り返す機能

① GPT3.5で簡単なタスク

In [5]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', openai_api_key=api_key)
tools = load_tools(['serpapi'], llm=llm, serpapi_api_key=serpapi_key)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
response = agent.run('2023 WORLD BASEBALL CLASSICの大谷翔平の成績をまとめて下さい。')



> Entering new AgentExecutor chain...
I may need to search for information on the 2023 World Baseball Classic and Shohei Ohtani's performance.
Action: Search
Action Input: "2023 World Baseball Classic Shohei Ohtani stats"
Observation: He hit . 435/. 606/. 739 with four doubles, a home run, and 10 walks in the tournament while also pitching to a 1.86 ERA with 11 strikeouts in 9 2/3 innings.
Thought:That's a great performance by Shohei Ohtani!
Thought: I now know the final answer
Final Answer: Shohei Ohtani hit .435/.606/.739 with four doubles, a home run, and 10 walks in the 2023 World Baseball Classic, while also pitching to a 1.86 ERA with 11 strikeouts in 9 2/3 innings.

> Finished chain.


② GPT3.5でちょいムズなタスク

In [6]:
# llm = ChatOpenAI(model_name='gpt-3.5-turbo', openai_api_key=api_key)
# tools = load_tools(['serpapi'], llm=llm, serpapi_api_key=serpapi_key)
# agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
# response = agent.run('2023 WORLD BASEBALL CLASSICの大谷翔平の成績をマークダウン形式で表にまとめて下さい。')

③ GPT4でちょいムズなタスク

In [7]:
llm = ChatOpenAI(model_name='gpt-4', openai_api_key=api_key)
tools = load_tools(['serpapi'], llm=llm, serpapi_api_key=serpapi_key)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
response = agent.run('2023 WORLD BASEBALL CLASSICの大谷翔平の成績をマークダウン形式で表にまとめて下さい。')



> Entering new AgentExecutor chain...
まず、2023年のワールド・ベースボール・クラシック(WBC)が開催されたかどうかと、大谷翔平の成績を調べる必要がある。
Action: Search
Action Input: 2023 World Baseball Classic Shohei Ohtani stats
Observation: He hit . 435/. 606/. 739 with four doubles, a home run, and 10 walks in the tournament while also pitching to a 1.86 ERA with 11 strikeouts in 9 2/3 innings.
Thought:2023年のワールド・ベースボール・クラシックで大谷翔平の成績を調べたので、マークダウン形式で表にまとめる。

Final Answer:

```
| 項目            | 成績           |
|-----------------|----------------|
| 打率            | .435           |
| 出塁率          | .606           |
| 長打率          | .739           |
| 二塁打          | 4              |
| ホームラン      | 1              |
| 四球            | 10             |
| 投球イニング    | 9 2/3          |
| 防御率          | 1.86           |
| 奪三振          | 11             |
```

> Finished chain.


<font color="RoyalBlue">結論：Agentsを動かすにはGPT3.5はアホすぎる</font>

# 会話を続けられて検索もできるエージェントを作る

In [8]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm = ChatOpenAI(model_name='gpt-4', openai_api_key=api_key)
tools = load_tools(['serpapi'], llm=llm, serpapi_api_key=serpapi_key)
agent_chain = initialize_agent(tools, llm, agent='conversational-react-description', verbose=True, memory=memory)

In [9]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[]), output_key=None, input_key=None, return_messages=True, human_prefix='Human', ai_prefix='AI', memory_key='chat_history')

In [10]:
agent_chain.run('2023 WORLD BASEBALL CLASSICの大谷翔平の成績をまとめて下さい。')



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: 2023 World Baseball Classic Shohei Ohtani stats
Observation: He hit . 435/. 606/. 739 with four doubles, a home run, and 10 walks in the tournament while also pitching to a 1.86 ERA with 11 strikeouts in 9 2/3 innings.
Thought:Do I need to use a tool? No
AI: 大谷翔平選手の2023年のワールド・ベースボール・クラシック成績は、打率.435、出塁率.606、長打率.739で、ダブル4本、ホームラン1本、四球10個を記録しました。また、投手としては、9 2/3イニングで11奪三振、防御率1.86の成績を残しています。

> Finished chain.


'大谷翔平選手の2023年のワールド・ベースボール・クラシック成績は、打率.435、出塁率.606、長打率.739で、ダブル4本、ホームラン1本、四球10個を記録しました。また、投手としては、9 2/3イニングで11奪三振、防御率1.86の成績を残しています。'

In [11]:
agent_chain.run('彼は何本のホームランを打ちましたか？')



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: 大谷翔平選手は、2023年のワールド・ベースボール・クラシックで1本のホームランを打ちました。

> Finished chain.


'大谷翔平選手は、2023年のワールド・ベースボール・クラシックで1本のホームランを打ちました。'

In [12]:
memory.clear()